In [1]:
import json

In [3]:
with open('../data/WikiHow_procedural_concrete_question.json', 'r') as f:
    concrete_question = json.load(f)

with open('../data/WikiHow_id2question.json', 'r') as f:
    id2question = json.load(f)

with open('../data/WikiHow_question2id.json', 'r') as f:
    question2id = json.load(f)

data = []
with open('../data/WikiHowNFQA.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
        
with open('../data/WikiHow_procedural_concrete_question.json', 'r') as f:
    results = json.load(f)

In [24]:
index = 'wikihow_16'
print(id2question[index])
print(results[index])

How To Tune a Bass Guitar
1. Turn on the tuner and play the first string on the guitar.
2. Adjust the corresponding tuning peg until the tuner lights up green, then repeat with the rest of the strings.
3. If you don't have a tuner, play the corresponding notes on a tuned instrument.
4. Adjust the guitar strings with the tuning pegs until the notes match the other instrument.
